In [2]:
import numpy as np
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize, curve_order, final_exponentiate, eq
import galois

print("Initializing a large field...")
GF = galois.GF(curve_order)
print("Field initialized")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Initializing a large field...
Field initialized


# Problem Definition

In [140]:
# define the matrices in our Galois Field
L = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]))
R = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]))


O = GF(np.array([
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, curve_order-1, curve_order-1, 0, 0, 0]]))

# witness vector `a` as named in the paper
a = GF(np.array([1, 169695, 3, 12, 7, 9, 27, 324, 144, 1728, 98]))

# Get u, v, w
def interpolate_column(col, nb):
    xs = GF(np.arange(1,nb+1))
    return galois.lagrange_poly(xs, col)

def get_polys_of_matrix(matrix):
    """
    Interpolates each column of matrix into a polynomial.
    Returns a list of polynomials of columns.
    """
    polys = []
    nb_of_rows = len(matrix)
    nb_of_columns = len(matrix[0])
    # for each column
    for col_id in range(nb_of_columns):
        column = []
        for row in range(nb_of_rows):
            column.append(matrix[row][col_id])
        polys.append(interpolate_column(GF(np.array(column)), nb_of_rows))
    return np.array(polys)
    
## computes all interpolated polynomials for L, R, and O
u = get_polys_of_matrix(L)
v = get_polys_of_matrix(R)
w = get_polys_of_matrix(O)

# Circuit dependent vars
n = len(L)
l = 2    # number of public vars. Only [1, out, <private> ]
m = len(L[0]) - 1
assert m == len(a) - 1, "Matrix and s vector do not match"

# t = (x-1)(x-2)...(x-n)
t = galois.Poly([1], field=GF)
for i in range(1, n+1):
    t = t * galois.Poly([1, curve_order-i], field=GF)


# Setup
Calculate $\sigma$ and $\tau$. We need to pick arbitrary generators G and H for elliptic curve groups and also randomly pick $\alpha, \beta, \gamma, \delta, x$ from the related galois fields. Then we can compute the following:

$\sigma = ( G^\alpha, G^\beta, H^\beta, H^\gamma, G^\delta, H^\delta, \{G^{x^i}\}_{i=0}^{n-1}, \{H^{x^i}\}_{i=0}^{n-1},\{G^{\frac{\beta u_i(x)+\alpha v_i(x) + w_i(x)}{\gamma}}\}_{i=0}^{l},\{G^{\frac{\beta u_i(x)+\alpha v_i(x) + w_i(x)}{\delta}}\}_{i=l+1}^{m},\{G^{\frac{x^i t(x)}{\delta}}\}_{i=0}^{n-2} )$


$\tau = ( \alpha, \beta, \gamma, \delta, x )$

Beware that $x$ is fixed.

In [141]:
class Sigma:
    G_alpha = None
    G_beta = None
    H_beta = None
    H_gamma = None
    G_delta = None
    H_delta = None
    G_x_powers = None
    H_x_powers = None
    G_public = None
    G_private = None
    G_end = None  # Could not find a better name
    
    def __init__(self, G, H, alpha, beta, gamma, delta, x, n, l, m, u, v, w, t):
        self.G_alpha = multiply(G, int(alpha))
        self.G_beta = multiply(G, int(beta))
        self.H_beta = multiply(H, int(beta))
        self.H_gamma = multiply(H, int(gamma))
        self.G_delta = multiply(G, int(delta))
        self.H_delta = multiply(H, int(delta))
        self.G_x_powers = [multiply(G, int(x**i)) for i in range(n)] # from 0 to n-1 inclusive
        self.H_x_powers = [multiply(H, int(x**i)) for i in range(n)] # from 0 to n-1 inclusive
              
        self.G_public = []
        for i in range(l+1):  # from 0 to l, inclusive
            numerator = Sigma._numerator(beta, u[i], alpha, v[i], w[i], x)
            exponent = numerator / gamma
            self.G_public.append(multiply(G, int(exponent)))
            
        self.G_private = [None] * (l+1)
        for i in range(l+1, m+1):  # from l+1 to m, inclusive
            numerator = Sigma._numerator(beta, u[i], alpha, v[i], w[i], x)
            exponent = numerator / delta
            self.G_private.append(multiply(G, int(exponent)))
        
        self.G_end = []
        for i in range(n-1):  # from 0 to n-2, inclusive
            exponent = x**i * t(x) / delta
            self.G_end.append(multiply(G, int(exponent)))

    @staticmethod
    def _numerator(beta, u_i, alpha, v_i, w_i, x):
        return beta * u_i(x) + alpha * v_i(x) + w_i(x)


class Tau:
    def __init__(self, alpha, beta, delta, gamma, x):
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.gamma = gamma
        self.x = x


class Relation:
    p = None # order of the finite field
    G = None # Generator of G_1 (changed from paper)
    H = None # Generator of G_2 (changed from paper)
    G_T = None # Unused or maybe set to FQ12.one() (changed from paper)
    e = None # pairing function
    l = None # number of public inputs
    u = None # List of u functions
    v = None # List of v functions
    w = None # List of w functions
    t = None # Part of balancing polynomial
    # Auxiliary (not defined as part of R in paper but related to R)
    m = None # Number of u/v/w functions minus 1 or number of columns of L/R/O minus 1
    n = None # Degree of t or degree of u/v/w functions plus 1
    
    def __init__(self, u, v, w, t):
        self.p = curve_order
        # TODO Why is this in the reverse order?
        self.G = G2
        self.H = G1
        self.e = pairing
        self.l = 2  # Witness vector: [1, out, <inputs, all private>]
        self.u = u
        self.v = v
        self.w = w
        self.t = t
        # Auxiliary (not defined as part of R in paper but related to R)
        self.m = len(u) - 1
        self.n = t.degree
        

def setup(rel: Relation):
    alpha = GF.Random()
    beta = GF.Random()
    gamma = GF.Random()
    delta = GF.Random()
    x = GF.Random() # Not witness, needed for Schwartz-Zippel lemma
    
    sigma = Sigma(rel.G, rel.H, alpha, beta, gamma, delta, x, 
                  rel.n, rel.l, rel.m, rel.u, rel.v, rel.w, rel.t)
    tau = Tau(alpha, beta, delta, gamma, x)  # We will not use
    return (sigma, tau)


# Set it up
rel = Relation(u, v, w, t)
sigma, tau = setup(rel)


# Prover

In [142]:
def calculate_h(rel: Relation, a):
    # Calculate h(x)
    sum_of_au = sum([a[i]*rel.u[i] for i in range(rel.m+1)], start=galois.Poly([0], field=GF))
    sum_of_av = sum([a[i]*rel.v[i] for i in range(rel.m+1)], start=galois.Poly([0], field=GF))
    sum_of_aw = sum([a[i]*rel.w[i] for i in range(rel.m+1)], start=galois.Poly([0], field=GF))
    h = (sum_of_au * sum_of_av - sum_of_aw) // rel.t
    return h

def prove_manual(rel: Relation, sigma: Sigma, tau: Tau, a):
    """
    This version manually calculate exponentiations using unknown-to-the-prover variables.
    ie. alpha, beta, ... 

    FIXME: Verifier does not work with this, seems to be bugged
    """
    r, s = GF.Random(), GF.Random()
    h = calculate_h(rel, a)
    A_exponent = tau.alpha + sum([a[i]*rel.u[i](tau.x) for i in range(rel.m+1)], start=GF(0)) + r*tau.delta
    B_exponent = tau.beta  + sum([a[i]*rel.v[i](tau.x) for i in range(rel.m+1)], start=GF(0)) + s*tau.delta
    C_exponent_term_1 = (sum([a[i] * (tau.beta*rel.u[i](tau.x)+tau.alpha*rel.v[i](tau.x)+rel.w[i](tau.x)) 
                              for i in range(rel.l+1, rel.m+1)]  # from l+1 to m
                             , start=GF(0))
                         + h(tau.x)*rel.t(tau.x) ) / tau.delta
    C_exponent_term_2 = s*(tau.alpha+sum([a[i]*rel.u[i](tau.x) for i in range(rel.m+1)], start=GF(0)))
    C_exponent_term_3 = r*(tau.beta +sum([a[i]*rel.v[i](tau.x) for i in range(rel.m+1)], start=GF(0)))
    C_exponent_term_4 = r*s*tau.delta
    A = multiply(G, int(A_exponent))
    B = multiply(H, int(B_exponent))
    C = multiply(G, int(C_exponent_term_1+C_exponent_term_2+C_exponent_term_3+C_exponent_term_4))
    return [A, B, C]

def polynomial_evaluation(polynomial, group_gen_x_powers, group_gen, n):
    """
    Evaluate the polynomial at x by multiplying G^{x^i} by itself coef_i many times.
    This allows computing the result without knowing x (and we should not know).

    n-1 is the degree of the polynomial

    TODO Maybe rename this rename this function? It is not necessarily evaluation.
    """
    coefs = polynomial.coefficients()
    poly_evaluation = multiply(group_gen, 0)
    # FIXME
    # assert n == len(group_gen_x_powers), "n is different than power count!"
    # NOTE Coefficients are descending but powers are ascending so we reverse coefs
    # NOTE If there are less coefficients than there are powers, then missing coefs are 0 => ignore
    for i, coef in enumerate(coefs[::-1]):
        poly_evaluation = add(poly_evaluation, multiply(group_gen_x_powers[i], int(coef)))
    return poly_evaluation

def sum_of_polynomial_evaluations(multipliers, polynomials, group_gen_x_powers, group_gen, n):
    summation = multiply(group_gen, 0)
    assert m+1 == len(polynomials), "Polynomial count error!"
    for i in range(len(polynomials)):
        polynomial = multipliers[i]*polynomials[i]
        evaluation = polynomial_evaluation(polynomial, group_gen_x_powers, group_gen, n)
        summation = add(summation, evaluation)
    return summation

def prove(rel: Relation, sigma, a):
    """
    FIXME Does not work again!
    """
    r, s = GF.Random(), GF.Random()
    h = calculate_h(rel, a)
    # Calculate A's term with sigma
    A_sum = sum_of_polynomial_evaluations(a, rel.u, sigma.G_x_powers, rel.G, rel.n)
    # Sum all terms of A
    A = add(add(sigma.G_alpha, A_sum), multiply(sigma.G_delta, int(r)))
    # Calculate B's term with sigma
    B_sum = sum_of_polynomial_evaluations(a, rel.v, sigma.H_x_powers, rel.H, rel.n)
    # Sum all terms of B
    B = add(add(sigma.H_beta, B_sum), multiply(sigma.H_delta, int(s)))
    # Calculate G^{h(x)*t(x)/delta} for C term 1
    G_ht_delta = polynomial_evaluation(h, sigma.G_end, G, rel.n-2)  # FIXME Is this correct?
    # Calculate C terms    
    C_term_1 = multiply(rel.G, 0)
    for i in range(l+1, m+1):  # Sigma part
        C_term_1 = add(C_term_1, multiply(sigma.G_private[i], int(a[i])))
    C_term_1 = add(C_term_1, G_ht_delta)
    # C term 2
    C_term_2 = add(sigma.G_alpha, sum_of_polynomial_evaluations(a, rel.u, sigma.G_x_powers, rel.G, rel.n))
    C_term_2 = multiply(C_term_2, int(s))
    # C term 3
    C_term_3 = add(sigma.G_beta, sum_of_polynomial_evaluations(a, rel.v, sigma.G_x_powers, rel.G, rel.n))
    C_term_3 = multiply(C_term_3, int(r))
    # C term 4
    C_term_4 = multiply(sigma.G_delta, int(r*s))
    # Finally, calculate C
    C = add(C_term_1, add(C_term_2, add(C_term_3, C_term_4)))
    return [A, B, C]

pi = prove(rel, sigma, a)
pi_man = prove_manual(rel, sigma, tau, a)
for i in range(len(pi)):
    print("Eqv of i:", eq(pi[i], pi_man[i]))


Eqv of i: False
Eqv of i: False
Eqv of i: False


# Verifier

In [143]:
def verify(rel: Relation, sigma, a, pi):
    A, B, C = pi
    pairing_lhs = rel.e(A, B)
    pairing_rhs_term_1 = rel.e(sigma.G_alpha, sigma.H_beta)

    #i = 0
    #pairing_rhs_term_2_left = multiply(sigma.G_public[i], int(a[i]))
    #for i in range(1, l+1):
    #    pairing_rhs_term_2_left = add(pairing_rhs_term_2_left, multiply(sigma.G_public[i], int(a[i])))
    #pairing_rhs_term_2_left_bak = pairing_rhs_term_2_left

    pairing_rhs_term_2_left = multiply(rel.G, 0)
    for i in range(rel.l+1):
        pairing_rhs_term_2_left = add(pairing_rhs_term_2_left, multiply(sigma.G_public[i], int(a[i])))

    #print(eq(pairing_rhs_term_2_left_bak, pairing_rhs_term_2_left))
    
    pairing_rhs_term_2 = rel.e(pairing_rhs_term_2_left, sigma.H_gamma)
    
    pairing_rhs_term_3 = rel.e(C, sigma.H_delta)
    return final_exponentiate(pairing_lhs) == final_exponentiate(pairing_rhs_term_1*pairing_rhs_term_2*pairing_rhs_term_3)

print(verify(rel, sigma, a, pi))
print(verify(rel, sigma, a, pi_man))

True
True
